# Dependencies

In [1]:
# Used Functions:
import cv2
import numpy as np
import os
import mediapipe as mp
from matplotlib import pyplot as plt
from tensorflow.keras.utils import to_categorical
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.callbacks import TensorBoard
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from sklearn.metrics import multilabel_confusion_matrix, accuracy_score
from scipy import stats
from glob import glob
from collections import Counter


# Developed Functions:
from Dev_Fun.Silhouette_Extraction import makeSegMask,sil_preprocess
from Dev_Fun.Joints_Extraction import mediapipe_detection, draw_landmarks
from Dev_Fun.Calibration_Show import Calib_Show
from Dev_Fun.Dataset_Capture import files_creation, dataset_capture
from Dev_Fun.Cropping_Image import crop_to_person
from Dev_Fun.Pre_Process import pre_process
from Dev_Fun.Pred_Show import pred_show

Using cache found in C:\Users\lance/.cache\torch\hub\pytorch_vision_v0.6.0
c:\Users\lance\AppData\Local\Programs\Python\Python311\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\lance\AppData\Local\Programs\Python\Python311\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=DeepLabV3_ResNet101_Weights.COCO_WITH_VOC_LABELS_V1`. You can also use `weights=DeepLabV3_ResNet101_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Model has been loaded.


Using cache found in C:\Users\lance/.cache\torch\hub\pytorch_vision_v0.6.0


Model has been loaded.


# Calibration
<img src="https://www.researchgate.net/profile/Jungpil-Shin/publication/362351225/figure/fig3/AS:1183521695645696@1659184969422/Mediapipe-detects-33-nodes-of-the-human-pose.ppm" width="400px">

In [2]:
#Feed Input Source
feed = 0
# Calib_Show(feed)

# Setup Folders

In [3]:
# Path for exported data, numpy arrays
DATA_PATH = os.path.join('Custom_Gait_Dataset_V.4')
EXPORTED = os.path.join('Exported')

# Persons that we try to detect
persons = np.array(['Lance Moheb','Seif Ibrahim','Mina Nady','Omar Amin','Mohmed Youssef','Hamza Tharwat'])
# persons = np.array(['Lance Moheb','Seif Ibrahim','Mina Nady','Omar Amin','Mohmed Youssef','Hamza Tharwat','Romario Nagy','Ziad Diaa'])

# 10 videos worth of data
no_sequences = 10

# Videos are going to be 30 frames in length
sequence_length = 90

persons_len = len(persons)
total_sequences = persons_len*no_sequences

# Capturing Dataset

In [4]:
# files_creation(persons,no_sequences,DATA_PATH)
# dataset_capture(persons,no_sequences,DATA_PATH,sequence_length,feed)

# Pre-Processing

* A) Importing Dataset

In [5]:
# label_map = {label:num for num, label in enumerate(persons)}
# print(label_map)

In [6]:
# sequences, labels = [], []
# for person in persons:
#     for sequence in np.array(os.listdir(os.path.join(DATA_PATH, person))).astype(int):
#         window = []
#         for frame_num in range(sequence_length):

#             image_path = os.path.join(DATA_PATH, person, str(sequence), f"{frame_num}.jpg")
#             print (image_path)
#             res = cv2.imread(image_path)
#             window.append(res)
#         sequences.append(window)
#         labels.append(label_map[person])

In [7]:
# seqq=np.array(sequences)
# print(seqq.shape)
# show_test = cv2.cvtColor(seqq[0][30], cv2.COLOR_BGR2RGB)
# plt.imshow(show_test)

* B) dataset preproccesing
    * Flip Horizontal
    * Converting to Silhouette
        * Background Subtraction
        * Noise Reduction
        * Binarization
    * convert to gray-scale
    * Resize to CASIA-B Resolution (320x240)

In [8]:
# proccessed = []
# fgbg = cv2.createBackgroundSubtractorKNN(detectShadows=False)
# for sequence in  range(total_sequences):
# # for sequence in  range(1):
#     for frame_num in range(sequence_length):

#         frame = cv2.cvtColor(seqq[sequence][frame_num], cv2.COLOR_BGR2RGB)

#         flip = cv2.flip(frame, 1)

#         sil = makeSegMask(flip)

#         gray = cv2.cvtColor(sil, cv2.COLOR_BGR2GRAY)

#         resize = cv2.resize(gray,(320,240))

#         crop = crop_to_person(resize)

#         proccessed.append(crop)
        
#         print (len(proccessed))

In [9]:
# plt.imshow(proccessed[30],cmap='gray')

* C) Export the Custom dataset with CASIA-B File Structure

In [10]:
# def create_directories_if_not_exist(path):
#     if not os.path.exists(path):
#         os.makedirs(path)

In [11]:
# # Define your dataset path
# dataset_path = "CASIA-B"

# # Define walking conditions
# walking_conditions = [
#     "bg-01", "bg-02",
#     "cl-01", "cl-02",
#     "nm-01", "nm-02", "nm-03", "nm-04", "nm-05", "nm-06"
# ]

# # Iterate over subjects
# for subject_id in range(119, 128):
#     subject_folder = os.path.join(dataset_path, str(subject_id))

#     # Iterate over walking conditions
#     for walking_condition in walking_conditions:
#         walking_condition_folder = os.path.join(subject_folder, walking_condition,str("090"))

#         # Iterate over frames
#         for frame_num in range(90):
#             # Define frame name
#             frame_name = f"{subject_id}-{walking_condition}-090-{frame_num:03d}.png"
#             frame_path = os.path.join(walking_condition_folder, frame_name)

#             walking_condition_index = walking_conditions.index(walking_condition)
#             index = (subject_id - 119) * len(walking_conditions) * 90 + walking_conditions.index(walking_condition) * 90 + frame_num
#             print(index)

#             # Check if the image is empty
#             if proccessed[index] is None:
#                 # Create a white image
#                 image = np.ones((128, 64, 3), dtype=np.uint8) * 255  # Assuming 128x64 is the size of your image
#             else:
#                 image = proccessed[index]

#             create_directories_if_not_exist(walking_condition_folder)
#             cv2.imwrite(frame_path, image)
#             print(f"Exported: {frame_path}")

# print("Images exported successfully!")


* D) Pre-Proccesing CASIA-B

In [12]:
def import_images(input):
  image_pattern = os.path.join(input, "*.png")
  image_paths = glob(image_pattern)

  return image_paths

In [13]:
# # Define your dataset path
# dataset_path = "CASIA-B"
# export_path = "exported"

# # Define walking conditions
# walking_conditions = [
#     "bg-01", "bg-02",
#     "cl-01", "cl-02",
#     "nm-01", "nm-02", "nm-03", "nm-04", "nm-05", "nm-06"
# ]

# # Iterate over subjects
# for subject_id in range(1,119):
#     subject_folder = os.path.join(dataset_path, f"{subject_id:03d}")
#     subject_folder_export = os.path.join(export_path, f"{subject_id:03d}")

#     # Iterate over walking conditions
#     for walking_condition in walking_conditions:
#         walking_condition_folder = os.path.join(subject_folder, walking_condition,str("090"))
#         walking_condition_folder_export = os.path.join(subject_folder_export, walking_condition,str("090"))
#         create_directories_if_not_exist(walking_condition_folder_export)
        
#         image_list = import_images(walking_condition_folder)
#         for image_path in image_list:

#             img = cv2.imread(image_path)
#             gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
#             crop = crop_to_person(gray)

#             new_path = image_path.replace("CASIA-B", "exported")

#             if crop is None:
#                 break
#             else:
#                 cv2.imwrite(new_path, crop)

#             print(new_path, "###DONE###")
# print("Images exported successfully!")


# Train

A) Splitting Dataset

In [14]:
subjects = np.array([f"{i:03d}" for i in range(1, 125)])
# subjects = np.concatenate((numbers_array, persons))
print(len(subjects))
print(subjects)


124
['001' '002' '003' '004' '005' '006' '007' '008' '009' '010' '011' '012'
 '013' '014' '015' '016' '017' '018' '019' '020' '021' '022' '023' '024'
 '025' '026' '027' '028' '029' '030' '031' '032' '033' '034' '035' '036'
 '037' '038' '039' '040' '041' '042' '043' '044' '045' '046' '047' '048'
 '049' '050' '051' '052' '053' '054' '055' '056' '057' '058' '059' '060'
 '061' '062' '063' '064' '065' '066' '067' '068' '069' '070' '071' '072'
 '073' '074' '075' '076' '077' '078' '079' '080' '081' '082' '083' '084'
 '085' '086' '087' '088' '089' '090' '091' '092' '093' '094' '095' '096'
 '097' '098' '099' '100' '101' '102' '103' '104' '105' '106' '107' '108'
 '109' '110' '111' '112' '113' '114' '115' '116' '117' '118' '119' '120'
 '121' '122' '123' '124']


In [15]:
label_map = {label:num for num, label in enumerate(subjects)}
print(label_map)

{'001': 0, '002': 1, '003': 2, '004': 3, '005': 4, '006': 5, '007': 6, '008': 7, '009': 8, '010': 9, '011': 10, '012': 11, '013': 12, '014': 13, '015': 14, '016': 15, '017': 16, '018': 17, '019': 18, '020': 19, '021': 20, '022': 21, '023': 22, '024': 23, '025': 24, '026': 25, '027': 26, '028': 27, '029': 28, '030': 29, '031': 30, '032': 31, '033': 32, '034': 33, '035': 34, '036': 35, '037': 36, '038': 37, '039': 38, '040': 39, '041': 40, '042': 41, '043': 42, '044': 43, '045': 44, '046': 45, '047': 46, '048': 47, '049': 48, '050': 49, '051': 50, '052': 51, '053': 52, '054': 53, '055': 54, '056': 55, '057': 56, '058': 57, '059': 58, '060': 59, '061': 60, '062': 61, '063': 62, '064': 63, '065': 64, '066': 65, '067': 66, '068': 67, '069': 68, '070': 69, '071': 70, '072': 71, '073': 72, '074': 73, '075': 74, '076': 75, '077': 76, '078': 77, '079': 78, '080': 79, '081': 80, '082': 81, '083': 82, '084': 83, '085': 84, '086': 85, '087': 86, '088': 87, '089': 88, '090': 89, '091': 90, '092': 9

In [16]:
# Define your dataset path
dataset_path = "Custom_Gait_Dataset_V.5"
sequneces_train, sequneces_test, labels_train, labels_test  = [], [], [], []

# Define walking conditions
walking_conditions_train = [
    "nm-01", "nm-02", "nm-03", "nm-04"
]
walking_conditions_test = [
    "nm-05", "nm-06"
]

error_num = 0
error = []


# ---Taining Set---
for subject_id in subjects:
    subject_folder = os.path.join(dataset_path, subject_id)

    set = []
    # Iterate over walking conditions
    for walking_condition in walking_conditions_train:
        walking_condition_folder = os.path.join(subject_folder, walking_condition,str("090"))
        
        window = []

        image_list = import_images(walking_condition_folder)
        for image_path in image_list[:30]:
            img = cv2.imread(image_path)
            gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
            print(image_path)
            window.append(gray)
        print("------------------------------------------")
        # set.append(window)
        labels_train.append(label_map[str(subject_id)])
        sequneces_train.append(window)

        if(len(window)!= 30):
            error_num += 1
            error.append(subject_id)
    print("#################################################################")
    

# ---Testing Set---
for subject_id in subjects:
    subject_folder = os.path.join(dataset_path, subject_id)

    set = []
    # Iterate over walking conditions
    for walking_condition in walking_conditions_test:
        walking_condition_folder = os.path.join(subject_folder, walking_condition,str("090"))
        
        window = []

        image_list = import_images(walking_condition_folder)
        for image_path in image_list[:30]:
            img = cv2.imread(image_path)
            gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
            print(image_path)
            window.append(gray)
        print("------------------------------------------")
        # set.append(window)
        labels_test.append(label_map[str(subject_id)])
        sequneces_test.append(window)

        if(len(window)!= 30):
            error_num += 1
            error.append(subject_id)
    print("#################################################################")
    
print("Images loaded successfully!")
print("errors " + str(error_num))
print("error list:  " + str(error))


Custom_Gait_Dataset_V.5\001\nm-01\090\001-nm-01-090-047.png
Custom_Gait_Dataset_V.5\001\nm-01\090\001-nm-01-090-048.png
Custom_Gait_Dataset_V.5\001\nm-01\090\001-nm-01-090-049.png
Custom_Gait_Dataset_V.5\001\nm-01\090\001-nm-01-090-050.png
Custom_Gait_Dataset_V.5\001\nm-01\090\001-nm-01-090-051.png
Custom_Gait_Dataset_V.5\001\nm-01\090\001-nm-01-090-052.png
Custom_Gait_Dataset_V.5\001\nm-01\090\001-nm-01-090-053.png
Custom_Gait_Dataset_V.5\001\nm-01\090\001-nm-01-090-054.png
Custom_Gait_Dataset_V.5\001\nm-01\090\001-nm-01-090-055.png
Custom_Gait_Dataset_V.5\001\nm-01\090\001-nm-01-090-056.png
Custom_Gait_Dataset_V.5\001\nm-01\090\001-nm-01-090-057.png
Custom_Gait_Dataset_V.5\001\nm-01\090\001-nm-01-090-058.png
Custom_Gait_Dataset_V.5\001\nm-01\090\001-nm-01-090-059.png
Custom_Gait_Dataset_V.5\001\nm-01\090\001-nm-01-090-060.png
Custom_Gait_Dataset_V.5\001\nm-01\090\001-nm-01-090-061.png
Custom_Gait_Dataset_V.5\001\nm-01\090\001-nm-01-090-062.png
Custom_Gait_Dataset_V.5\001\nm-01\090\00


Custom_Gait_Dataset_V.5\022\nm-02\090\022-nm-02-090-068.png
Custom_Gait_Dataset_V.5\022\nm-02\090\022-nm-02-090-069.png
Custom_Gait_Dataset_V.5\022\nm-02\090\022-nm-02-090-070.png
Custom_Gait_Dataset_V.5\022\nm-02\090\022-nm-02-090-071.png
Custom_Gait_Dataset_V.5\022\nm-02\090\022-nm-02-090-072.png
Custom_Gait_Dataset_V.5\022\nm-02\090\022-nm-02-090-073.png
Custom_Gait_Dataset_V.5\022\nm-02\090\022-nm-02-090-074.png
Custom_Gait_Dataset_V.5\022\nm-02\090\022-nm-02-090-075.png
Custom_Gait_Dataset_V.5\022\nm-02\090\022-nm-02-090-076.png
Custom_Gait_Dataset_V.5\022\nm-02\090\022-nm-02-090-077.png
Custom_Gait_Dataset_V.5\022\nm-02\090\022-nm-02-090-078.png
Custom_Gait_Dataset_V.5\022\nm-02\090\022-nm-02-090-079.png
Custom_Gait_Dataset_V.5\022\nm-02\090\022-nm-02-090-080.png
Custom_Gait_Dataset_V.5\022\nm-02\090\022-nm-02-090-081.png
Custom_Gait_Dataset_V.5\022\nm-02\090\022-nm-02-090-082.png
------------------------------------------
Custom_Gait_Dataset_V.5\022\nm-03\090\022-nm-03-090-052.

In [17]:
x_train = np.array(sequneces_train)
x_test = np.array(sequneces_test)
y_train = to_categorical(labels_train).astype(int)
y_test = to_categorical(labels_test).astype(int)

In [18]:
print(x_train.shape)
print(x_test.shape)
print(y_train.shape)
print(y_test.shape)

(496, 30, 64, 44)
(248, 30, 64, 44)
(496, 124)
(248, 124)


B) Building CNN Architecture

In [19]:
log_dir = os.path.join('Logs')
tb_callback = TensorBoard(log_dir=log_dir)

In [20]:
def cnn(input_shape = (30, 64, 44), num_classes = y_train.shape[1]):
    model = Sequential()
    model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=input_shape))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    
    model.add(Flatten())

    model.add(Dense(512, activation='relu'))

    model.add(Dense(num_classes, activation='softmax'))

    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

In [21]:
model = cnn()

In [22]:
model.fit(x_train, y_train, batch_size=32 , epochs=200, callbacks=[tb_callback])

Epoch 1/200


16/16 [==============================] - 1s 18ms/step - loss: 66.3472 - accuracy: 0.0081
Epoch 2/200
16/16 [==============================] - 0s 16ms/step - loss: 5.0990 - accuracy: 0.0081
Epoch 3/200
16/16 [==============================] - 0s 17ms/step - loss: 4.6040 - accuracy: 0.0605
Epoch 4/200
16/16 [==============================] - 0s 17ms/step - loss: 3.7892 - accuracy: 0.2016
Epoch 5/200
16/16 [==============================] - 0s 16ms/step - loss: 2.3825 - accuracy: 0.4960
Epoch 6/200
16/16 [==============================] - 0s 17ms/step - loss: 0.9029 - accuracy: 0.8024
Epoch 7/200
16/16 [==============================] - 0s 16ms/step - loss: 0.2671 - accuracy: 0.9496
Epoch 8/200
16/16 [==============================] - 0s 15ms/step - loss: 0.0708 - accuracy: 0.9859
Epoch 9/200
16/16 [==============================] - 0s 17ms/step - loss: 0.0217 - accuracy: 0.9980
Epoch 10/200
16/16 [==============================] - 0s 16ms/step - loss: 0.0060 - accuracy: 1.0

C) Training Results

In [23]:
print("Evaluate on test data")
results = model.evaluate(x_test, y_test)
print("test acc:", results[1])

Evaluate on test data
8/8 [==============================] - 0s 7ms/step - loss: 3.2732 - accuracy: 0.4556
test acc: 0.4556451737880707


In [24]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 28, 62, 32)        12704     
                                                                 
 max_pooling2d (MaxPooling2  (None, 14, 31, 32)        0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 12, 29, 64)        18496     
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 6, 14, 64)         0         
 g2D)                                                            
                                                                 
 conv2d_2 (Conv2D)           (None, 4, 12, 128)        73856     
                                                                 
 max_pooling2d_2 (MaxPoolin  (None, 2, 6, 128)         0

In [25]:
res = model.predict(x_test)
index = 2
tst = subjects[np.argmax(res[index])]
trn = subjects[np.argmax(y_test[index])]
print("Subject tested:" + tst + "\n" + "Subject trained:" + trn)

8/8 [==============================] - 0s 6ms/step
Subject tested:111
Subject trained:002


In [26]:
# model.save('train_55_6.h5')

In [27]:
yhat = model.predict(x_test)
ytrue = np.argmax(y_test, axis=1).tolist()
yhat = np.argmax(yhat, axis=1).tolist()

print(f"\n ##########< Accuracy: {(accuracy_score(ytrue, yhat)*100):.2f}% >########## \n")

print(multilabel_confusion_matrix(ytrue, yhat))


8/8 [==============================] - 0s 6ms/step

 ##########< Accuracy: 45.56% >########## 

[[[245   1]
  [  1   1]]

 [[246   0]
  [  2   0]]

 [[246   0]
  [  1   1]]

 [[246   0]
  [  0   2]]

 [[246   0]
  [  2   0]]

 [[246   0]
  [  1   1]]

 [[246   0]
  [  1   1]]

 [[244   2]
  [  2   0]]

 [[246   0]
  [  1   1]]

 [[243   3]
  [  2   0]]

 [[246   0]
  [  0   2]]

 [[246   0]
  [  2   0]]

 [[246   0]
  [  1   1]]

 [[245   1]
  [  2   0]]

 [[245   1]
  [  1   1]]

 [[246   0]
  [  1   1]]

 [[242   4]
  [  1   1]]

 [[245   1]
  [  1   1]]

 [[246   0]
  [  1   1]]

 [[246   0]
  [  1   1]]

 [[246   0]
  [  0   2]]

 [[246   0]
  [  1   1]]

 [[245   1]
  [  2   0]]

 [[246   0]
  [  1   1]]

 [[241   5]
  [  0   2]]

 [[245   1]
  [  1   1]]

 [[244   2]
  [  2   0]]

 [[246   0]
  [  1   1]]

 [[245   1]
  [  0   2]]

 [[246   0]
  [  2   0]]

 [[244   2]
  [  1   1]]

 [[246   0]
  [  1   1]]

 [[246   0]
  [  1   1]]

 [[244   2]
  [  0   2]]

 [[245   1]
  [  0  

# Detection

In [28]:
def Final_Pred(predictions):

    # Count the occurrences of each element
    counts = Counter(predictions)

    # Find the most common element
    most_common_element, count = counts.most_common(1)[0]

    percentage = (count / len(predictions)) * 100

    print("predictions array:", predictions)
    print("The most repeated element is:", most_common_element, "\nPercentage: ",percentage)

    return most_common_element

In [29]:
names = {
    "119" : "Lance Moheb",
    "120" : "Seif Ibrahim",
    "121" : "Mina Nady",
    "122" : "Omar Amin",
    "123" : "Mohmed Youssef",
    "124" : "Hamza Tharwat"
}

In [30]:
finalPred = pred_show('test6.mp4',model,names,subjects)
name = Final_Pred(finalPred)
print (name)

1/1 [==============================] - 0s 22ms/step
Omar Amin
1/1 [==============================] - 0s 20ms/step
Lance Moheb
1/1 [==============================] - 0s 20ms/step
026
##########< Unknown identity >##########
1/1 [==============================] - 0s 22ms/step
Omar Amin
1/1 [==============================] - 0s 15ms/step
Hamza Tharwat
1/1 [==============================] - 0s 21ms/step
Lance Moheb
1/1 [==============================] - 0s 14ms/step
026
##########< Unknown identity >##########
1/1 [==============================] - 0s 18ms/step
Seif Ibrahim
1/1 [==============================] - 0s 17ms/step
Omar Amin
1/1 [==============================] - 0s 19ms/step
Omar Amin
1/1 [==============================] - 0s 21ms/step
Seif Ibrahim
1/1 [==============================] - 0s 18ms/step
Seif Ibrahim
1/1 [==============================] - 0s 18ms/step
Omar Amin
1/1 [==============================] - 0s 20ms/step
Lance Moheb
1/1 [==============================] - 0s 2